In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/My\ Drive/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
 사진   celeba	'Colab Notebooks'   model


In [4]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torch.autograd import Variable
import torchvision

import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader


# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


In [0]:
# Root directory for dataset
dataroot = "/content/gdrive/My Drive/celeba/"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 50

# Learning rate for optimizers
lr = 0.0001

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [6]:
"""
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                           ]))

for num, value in enumerate(dataset):
    data, label = value
    data.save('/content/gdrive/My Drive/celeba/transformed/transformed_faces/%d.jpeg'%(num))

"""

"\ndataset = dset.ImageFolder(root=dataroot,\n                           transform=transforms.Compose([\n                               transforms.Resize(image_size),\n                               transforms.CenterCrop(image_size),\n                           ]))\n\nfor num, value in enumerate(dataset):\n    data, label = value\n    data.save('/content/gdrive/My Drive/celeba/transformed/transformed_faces/%d.jpeg'%(num))\n\n"

In [10]:
# We can use an image folder dataset the way we have it setup.
# Create the dataset


dataset = dset.ImageFolder(root='/content/gdrive/My Drive/celeba/transformed/',
                           transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                ]))

for num, value in enumerate(dataset):
    data, label = value
    torch.save(data, '/content/gdrive/My Drive/celeba/dataset/transform/%d.pt'%(num))

KeyboardInterrupt: ignored

In [18]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

dataset = dset.DatasetFolder(root='/content/gdrive/My Drive/celeba/dataset/', loader=dataloader, is_valid_file=pt)

NameError: ignored

In [0]:
dataset = dset.ImageFolder(root='/content/gdrive/My Drive/celeba/dataset/')

# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

In [0]:
# custom weights initialization called on netG and net_EN
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [0]:
# Generator Code

class DECODER(nn.Module):
    def __init__(self):
        super(DECODER, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [0]:
# Create the generator
net_DE = DECODER().cuda()


# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
net_DE.apply(weights_init)

# Print the model
print(net_DE)

In [0]:
class ENCODER(nn.Module):
    def __init__(self):
        super(ENCODER, self).__init__()
        self.conv = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(),
            # state size. (ndf*8) x 4 x 4
            )
        self.linear_mu = nn.Linear(8192, nz)
        self.linear_log = nn.Linear(8192, nz)

    def encode(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        mu = self.linear_mu(x)
        log_var = self.linear_log(x)
        return mu, log_var

    def reparametrize(self, mu, log_var):
        std = log_var.mul(0.5).exp_()
        eps = torch.FloatTensor(std.size()).normal_()
        eps = eps.cuda()
        return eps.mul(std).add_(mu)

    def forward(self, x):
        mu, log_var = self.encode(x)
        reparam = self.reparametrize(mu, log_var)
        return mu, log_var, reparam

In [0]:
# Create the Discriminator
net_EN = ENCODER().cuda()



# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
net_EN.apply(weights_init)

# Print the model
print(net_EN)

In [0]:
# Initialize BCELoss function
reconstruction_loss = nn.MSELoss()

def criteiron(recon_input, x, mu, log_var):
    BCE = reconstruction_loss(recon_input, x)
    KLD_element = mu.pow(2).add_(log_var.exp()).mul_(-1).add_(1).add_(log_var)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return BCE + KLD
# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1).cuda()


# Setup Adam optimizers for both G and D
parameters = list(net_EN.parameters())+ list(net_DE.parameters())
optimizer = torch.optim.Adam(parameters, lr=lr, betas=(beta1, 0.999))
#optimizer_EN = optim.Adam(net_EN.parameters(), lr=lr, betas=(beta1, 0.999))
#optimizer_DE = optim.Adam(net_DE.parameters(), lr=lr, betas=(beta1, 0.999))

In [0]:
# Training Loop

# Lists to keep track of progress
img_list = []
losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        #i [0]
        #data ([128,3,64,64],[128])

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        net_EN.zero_grad()
        net_DE.zero_grad()
        optimizer.zero_grad()
        # Format batch
        #real_cpu = data[0].cuda()
        x = data[0].cuda()
        #image = Variable(image).cuda()

        mu, log_var, reparam = net_EN(x)
        #[128,100,1,1][128,100,1,1][128,100,1,1]
        #print(type(output_EN), len(output_EN), output_EN.shape)

        #mu = mu.resize(128,100)
        #print(mu.shape)
        #log_var = log_var.view(128,100)
        reparam = reparam.view(-1,100,1,1)


        output = net_DE(reparam)
        #[128,3,64,64]

        err = criteiron(output, x, mu, log_var)
        err.backward()
        optimizer.step()

        # Output training stats
        if i % 10 == 0:
            print('[{} / {}] [{} / {}] Loss: {}'.format(epoch, num_epochs, i, len(dataloader), err))

        # Save Losses for plotting later
        losses.append(err.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = net_DE(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1   

In [0]:
plt.figure(figsize=(10,5))
plt.title("Loss During Training")
plt.plot(losses,label="Loss")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [0]:
#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [0]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].cuda()[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()